# **GC7 Model Inference**

Farhan Falahaqil Rafi

FTDS-003-BSD

# **Import Libraries**

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import pandas as pd
import torch

# **Model Loading**

In [3]:
# Load the fine-tuned model
model = BertForSequenceClassification.from_pretrained('./my_fine_tuned_bert')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\farha\Documents\GitHub\p2-ftds003-bsd-g7-7sugiwa\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ./my_fine_tuned_bert were not used when initializing BertForSequenceClassification: ['classifier.0.num_batches_tracked', 'classifier.0.running_var', 'classifier.0.bias', 'classifier.0.running_mean', 'classifier.1.weight', 'classifier.0.weight', 'classifier.1.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializ

# **Data Loading**

In [1]:
# List of headlines
headlines = [
    "Catholics Explain Why They Oppose Surrogacy",
    "Quiz: Who Said It, Donald Trump Or Hitler?",
    "Worst Pieces Of Sex Advice From Women's Magazines",
    "Republicans Explain Why They Don't Need Women Voters",
    "Americans Explain Who Jesus Would Vote For",
    "Christians Explain Why Atheists Are Bullies"
]

# Creating a DataFrame
df_headlines = pd.DataFrame(headlines, columns=['headline'])

# Displaying the DataFrame
print(df_headlines)

                                            headline
0        Catholics Explain Why They Oppose Surrogacy
1         Quiz: Who Said It, Donald Trump Or Hitler?
2  Worst Pieces Of Sex Advice From Women's Magazines
3  Republicans Explain Why They Don't Need Women ...
4         Americans Explain Who Jesus Would Vote For
5        Christians Explain Why Atheists Are Bullies


# **Model Inference**

In [4]:
# Tokenize the headlines in the DataFrame
inputs = tokenizer(df_headlines['headline'].tolist(), padding=True, truncation=True, max_length=256, return_tensors="pt")

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare the data for the model
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Perform inference
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Process the outputs to get predictions
predictions = torch.argmax(outputs.logits, dim=1)

# Map predictions to corresponding labels (e.g., 0 for 'Not Sarcastic', 1 for 'Sarcastic')
# (Adjust label names as per your original training labels)
predicted_labels = ["Sarcastic" if pred.item() == 1 else "Not Sarcastic" for pred in predictions]

# Add the predictions to the DataFrame
df_headlines['predicted_label'] = predicted_labels

# Display the DataFrame with predictions
print(df_headlines)

                                            headline predicted_label
0        Catholics Explain Why They Oppose Surrogacy   Not Sarcastic
1         Quiz: Who Said It, Donald Trump Or Hitler?   Not Sarcastic
2  Worst Pieces Of Sex Advice From Women's Magazines   Not Sarcastic
3  Republicans Explain Why They Don't Need Women ...   Not Sarcastic
4         Americans Explain Who Jesus Would Vote For       Sarcastic
5        Christians Explain Why Atheists Are Bullies   Not Sarcastic
